<a href="https://colab.research.google.com/github/Blind5518/pyspark-web-log-analysis/blob/main/pyspark_web_log_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Загрузка библиотек,  импорт и чтение данных**

In [12]:
# Усановка faker, pyspark, создание SparkSession
!pip install faker
!pip install pyspark


import csv
from faker import Faker
import random

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DateType, DoubleType, StructType, StructField, StringType, IntegerType


spark = SparkSession.builder \
    .appName("WebLogsAnalysis") \
    .getOrCreate()

# Показывать меньше логов spark
spark.sparkContext.setLogLevel("WARN")

**2. Генерация информации**

In [13]:
# Сгенерировали 100к записей логов и сохранили их в CSV-файл
fake = Faker()

num_records = 100000

http_methods = ['GET', 'POST', 'PUT', 'DELETE']
response_codes = [200, 301, 404, 500]

file_path = "/content/web_server_logs.csv"

with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ip', 'timestamp', 'method', 'url', 'response_code', 'response_size'])

    for _ in range(num_records):
        ip = fake.ipv4()
        timestamp = fake.date_time_this_year().isoformat()
        method = random.choice(http_methods)
        url = fake.uri_path()
        response_code = random.choice(response_codes)
        response_size = random.randint(100, 10000)

        writer.writerow([ip, timestamp, method, url, response_code, response_size])

print(f"Сгенерировано {num_records} записей и сохранено в {file_path}")

Сгенерировано 100000 записей и сохранено в /content/web_server_logs.csv


**3. Чтение и преобразование данных**

In [14]:
# 3) Чтение CSV
csv_path = "/content/web_server_logs.csv"

# Задаём схему для корректного чтения
schema = StructType([
    StructField("ip", StringType(), True),
    StructField("timestamp", StringType(), True),
    StructField("method", StringType(), True),
    StructField("url", StringType(), True),
    StructField("response_code", IntegerType(), True),
    StructField("response_size", IntegerType(), True),
])

df_raw = spark.read.csv(csv_path, header=True, schema=schema)
print("Прочитано строк:", df_raw.count())
df_raw.show(5, truncate=False)

Прочитано строк: 100000
+---------------+--------------------------+------+-------------------+-------------+-------------+
|ip             |timestamp                 |method|url                |response_code|response_size|
+---------------+--------------------------+------+-------------------+-------------+-------------+
|25.210.131.222 |2025-01-02T06:44:48.692879|DELETE|tags/tag/main      |500          |3279         |
|149.227.108.90 |2025-01-19T03:29:12.648483|GET   |tag/app/main       |301          |7916         |
|90.7.122.124   |2025-04-01T15:13:04.833127|GET   |app                |301          |2320         |
|103.243.108.112|2025-05-12T14:34:04.744041|POST  |category/categories|301          |7676         |
|95.156.99.82   |2025-08-04T23:20:15.968454|DELETE|list               |301          |4586         |
+---------------+--------------------------+------+-------------------+-------------+-------------+
only showing top 5 rows



In [15]:
# Приведение типов
df = df_raw.withColumn("timestamp_ts", F.to_timestamp("timestamp")) \
           .withColumn("date", F.to_date("timestamp_ts"))

# Проверим схему и первые строки
df.printSchema()
df.select("ip", "timestamp", "timestamp_ts", "date", "method", "response_code", "response_size").show(5, truncate=False)

root
 |-- ip: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: integer (nullable = true)
 |-- response_size: integer (nullable = true)
 |-- timestamp_ts: timestamp (nullable = true)
 |-- date: date (nullable = true)

+---------------+--------------------------+--------------------------+----------+------+-------------+-------------+
|ip             |timestamp                 |timestamp_ts              |date      |method|response_code|response_size|
+---------------+--------------------------+--------------------------+----------+------+-------------+-------------+
|25.210.131.222 |2025-01-02T06:44:48.692879|2025-01-02 06:44:48.692879|2025-01-02|DELETE|500          |3279         |
|149.227.108.90 |2025-01-19T03:29:12.648483|2025-01-19 03:29:12.648483|2025-01-19|GET   |301          |7916         |
|90.7.122.124   |2025-04-01T15:13:04.833127|2025-04-01 15:13:04.833127|2025-04-01|G

**4. Анализ полученных данных**

In [16]:
# 1) Сгруппировать по IP и показать 10 самых активных IP (по числу запросов).
top10_ips = (
    df.groupBy("ip")
      .agg(F.count("*").alias("request_count"))
      .orderBy(F.desc("request_count"))
      .limit(10)
)


# 2) Сгруппировать по HTTP-методу и посчитать количество запросов для каждого метода.
methods_count = (
    df.groupBy("method")
      .agg(F.count("*").alias("method_count"))
      .orderBy(F.desc("method_count"))
)


# 3) Количество запросов с кодом ответа 404.
count_404 = df.filter(F.col("response_code") == 404).count()


# 4) Сгруппировать данные по дате и просуммировать размер ответов, сортировать по дате.
bytes_per_date = (
    df.groupBy("date")
      .agg(F.sum("response_size").alias("total_response_size"),
           F.count("*").alias("method_count"))
      .orderBy("date")
)

In [17]:
print("\n1) Топ-10 самых активных IP (ip, requests_count):")
top10_ips.show(10, False)   # False (не усекать длинные строки)

print("\n2) Количество запросов по HTTP-методам (method, requests_count):")
methods_count.show(truncate=False)

print(f"\n3) Количество запросов с кодом ответа 404: {count_404}")

print("\n4) Сумма размеров ответов по дате (date, total_response_size):")
bytes_per_date.show(50, False)  # показываем первые n-дат


1) Топ-10 самых активных IP (ip, requests_count):
+--------------+-------------+
|ip            |request_count|
+--------------+-------------+
|222.10.27.122 |2            |
|57.98.92.105  |1            |
|99.237.97.97  |1            |
|179.42.169.227|1            |
|105.95.55.126 |1            |
|82.24.63.247  |1            |
|94.165.17.248 |1            |
|41.165.145.118|1            |
|44.177.75.76  |1            |
|65.117.232.214|1            |
+--------------+-------------+


2) Количество запросов по HTTP-методам (method, requests_count):
+------+------------+
|method|method_count|
+------+------------+
|PUT   |25250       |
|DELETE|25120       |
|POST  |24920       |
|GET   |24710       |
+------+------------+


3) Количество запросов с кодом ответа 404: 25097

4) Сумма размеров ответов по дате (date, total_response_size):
+----------+-------------------+------------+
|date      |total_response_size|method_count|
+----------+-------------------+------------+
|2025-01-01|1497018